In [1]:
import os,glob
import numpy as np
import pandas as pd
import glob
import pyslha
import time
import matplotlib.pyplot as plt
import os,glob
import numpy as np
import seaborn as sns
from scipy.special import kn,zetac
import pyslha
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
sns.set_palette(sns.color_palette("Paired"))

delphesDir = os.path.abspath("./DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT

ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')


Welcome to JupyROOT 6.24/06


True

In [13]:
inputFiles = ['./DelphesLLP/test_500.root','./DelphesLLP/test_700.root']

In [14]:
dists = {f : {} for f in inputFiles}

for f in inputFiles:
    totalWeight = 0.0
    ff = ROOT.TFile(f,'read')
    tree = ff.Get("Delphes")
    nevts = tree.GetEntries()
    for ievt in range(nevts):    
        tree.GetEntry(ievt)  
        totalWeight += tree.Event.At(0).Weight 

        topDaughters = []
        for p in tree.Particle:
            if abs(p.PID) != 6:
                continue
            if p.Status != 62:
                continue
            
            for dd in range(p.D1,p.D2+1):
                d = tree.Particle.At(dd)
                if abs(d.PID) == 24:
                    dlist = list(range(d.D1,d.D2+1))
                    while len(dlist) == 1:
                        d = tree.Particle.At(dlist[0])
                        dlist = list(range(d.D1,d.D2+1))

                    for dd2 in range(d.D1,d.D2+1):
                        d2 = tree.Particle.At(dd2)
                        topDaughters.append(abs(d2.PID))
                else:
                    topDaughters.append(abs(d.PID))
        
        finalStates = tuple(sorted(topDaughters))
        if finalStates not in dists[f]:
            dists[f][finalStates] = 0
        dists[f][finalStates] += 1  

    ff.Close()      
    print(f,'Total weight = %1.3e' %totalWeight)    



./DelphesLLP/test_500.root Total weight = -6.125e+02
./DelphesLLP/test_700.root Total weight = -3.505e+02


In [17]:

for f in dists:
    print(f)
    nhadronic = 0
    nsemileptonic = 0
    nleptonic = 0
    for fs in sorted(dists[f].keys()):
        if sum(fs.count(x) for x in [11,13,15]) == 0:
            nhadronic += float(dists[f][fs])
        elif sum(fs.count(x) for x in [11,13,15]) == 1:
            nsemileptonic += float(dists[f][fs])
        elif sum(fs.count(x) for x in [11,13,15]) == 2:
            nleptonic += float(dists[f][fs])
    nevts = nhadronic+nsemileptonic+nleptonic
    print(nevts)
    print('Hadronic = %1.3f' %(nhadronic/nevts)) # expected = 0.6741*0.6741 = 0.4544
    print('Semi-leptonic = %1.3f' %(nsemileptonic/nevts))  # expected = 2*0.6741*(1-0.6741) = 0.439378
    print('Leptonic = %1.3f' %(nleptonic/nevts)) # expected = (1-0.6741)*(1-0.6741) = 0.1062

./DelphesLLP/test_500.root
10000.0
Hadronic = 0.477
Semi-leptonic = 0.427
Leptonic = 0.095
./DelphesLLP/test_700.root
10000.0
Hadronic = 0.470
Semi-leptonic = 0.429
Leptonic = 0.101
